In [1]:
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.features as features
import numpy as np
from rasterio.enums import Resampling
from rasterio.crs import CRS
import itertools

from shapely.geometry import box

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
kdprov = '33'

In [3]:
popdens_100m=gpd.read_file("data/output/1_population_density/grid_pop_"+kdprov+"_100m.gpkg")

In [4]:
popdens_100m

,DN_left,iddesa,TOTAL_PEND,p_area,j_pend,geometry
0,19.0,3320120001,5003,0.006319,31.611906,"POLYGON ((11024100.000 -716100.000, 11024200.0..."
1,6.0,3320120001,5003,0.001995,9.982707,"POLYGON ((11024200.000 -716100.000, 11024300.0..."
2,227.0,3320120001,5003,0.075491,377.679082,"POLYGON ((11023900.000 -716200.000, 11024000.0..."
3,840.0,3320120001,5003,0.279348,1397.578982,"POLYGON ((11024000.000 -716200.000, 11024100.0..."
4,267.0,3320120001,5003,0.088793,444.230462,"POLYGON ((11024100.000 -716200.000, 11024200.0..."
...,...,...,...,...,...,...
1054075,3142.0,3312010003,4456,0.026323,117.293612,"POLYGON ((11035400.000 -1000400.000, 11035500...."
1054076,25.0,3312010003,4456,0.000209,0.933272,"POLYGON ((11035500.000 -1000400.000, 11035600...."
1054077,450.0,3312010003,4456,0.003770,16.798894,"POLYGON ((11035300.000 -1000500.000, 11035400...."
1054078,1777.0,3312010003,4456,0.014887,66.336966,"POLYGON ((11035400.000 -1000500.000, 11035500...."


In [5]:
popdens_100m.j_pend.sum()

34148421.99999999

In [6]:
popdens_100m['j_pend_round']=popdens_100m.j_pend.apply(lambda y: np.round(y))

In [7]:
raster = rasterio.open(r"/vsigs/bps-gcp-bucket/Degree of Urbanisation/GHSL data/2020/prov/2020_"+kdprov+".tif")
geom_value = ((geom,value) for geom, value in zip(popdens_100m.geometry, popdens_100m['j_pend_round']))
rasterized = features.rasterize(geom_value,
                                out_shape = raster.shape,
                                transform = raster.transform,
                                all_touched = False,
                                fill = 0,
                                dtype = np.int32)
with rasterio.open(
        "data/temp/rasterized_popdens_"+kdprov+"_100.tif", "w",
        driver = "GTiff",
        transform = raster.transform,
        dtype = np.int32,
        count = 1,
        width = raster.width,
        height = raster.height) as dst:
    dst.write(rasterized, indexes = 1)

In [8]:
import rasterio
from rasterio.enums import Resampling

target_res = (1000, 1000)

with rasterio.open("data/temp/rasterized_popdens_"+kdprov+"_100.tif") as src:
    data, transform = rasterio.warp.reproject(source=src.read(),
                                src_transform=src.transform,
                                src_crs=CRS.from_string("ESRI:54009"),
                                dst_crs=CRS.from_string("ESRI:54009"),
                                dst_nodata=0,
                                dst_resolution=target_res,
                                              #dtype = np.float64
                                resampling=Resampling.sum)
    profile = src.profile
    profile.update(transform=transform, driver='GTiff',
                    height=data.shape[1], width=data.shape[2])

    with rasterio.open('data/output/1_population_density/grid_pop_'+kdprov+'_1km.tif', 'w', **profile) as dst:
                    dst.write(data)

In [9]:
with rasterio.open('data/output/1_population_density/grid_pop_'+kdprov+'_1km.tif') as dataset:
    data = dataset.read(1)

    t = dataset.transform
    move_x = t[0]
    move_y = t[4]
    height = dataset.height
    width = dataset.width 
    
    polygons = []
    indices = list(itertools.product(range(width), range(height)))
    for x,y in indices:
        x_min, y_max = t * (x,y)
        x_max = x_min + move_x
        y_min = y_max + move_y
        polygons.append(box(x_min, y_min, x_max, y_max))
        
data_list = []
for x,y in indices:
    data_list.append(data[y,x])
    
vect_tmp = gpd.GeoDataFrame(data=data_list, geometry=polygons, columns=['DN'])
vect_tmp.crs='ESRI:54009'
vect_temp=vect_tmp.loc[vect_tmp.DN > -1]


In [10]:
data_builtup=gpd.read_file("data/input/GHSL Data/2020/gpkg/ghs_built_"+kdprov+"_nonzero.shp")#.to_crs(('ESRI:54009'))

In [11]:
data_builtup['builtup']=data_builtup['DN']

In [12]:
vect_=vect_temp.reset_index()
vect_['index_o']=vect_['index'].apply(lambda y:str(y).zfill(7))

In [13]:
data_bu_per1km=vect_.sjoin(data_builtup).sort_values('index_o')[['index_o','builtup']].groupby('index_o').agg('sum').reset_index()

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: ESRI:54009
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


In [14]:
vect_bu=vect_.merge(data_bu_per1km,how='left')
vect_bu['p_builtup']=vect_bu.builtup/1e6
vect_bu['pop']=vect_bu.DN

In [15]:
vect_bu=vect_bu[['geometry','p_builtup','pop']].fillna(0)
vect_bu.crs='ESRI:54009'

In [18]:
vect_bu.query('(pop>0) or (p_builtup>0)').to_file("data/output/1_population_density/grid_pop_"+kdprov+"_1km_w_bu.gpkg",driver='GPKG')